# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [37]:
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [38]:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [39]:
salaries.sample(10)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
7150,"DIAZ, JACQUELINE",HEAD LIBRARY CLERK,PUBLIC LIBRARY,F,Salary,NaN,60972.0,NaN
22811,"PENNICK II, MICKEY D",OPERATING ENGINEER-GROUP A,WATER MGMNT,F,Hourly,40.0,NaN,47.44
9293,"FOULKS, JENNIFER A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
22066,"ORTIZ, PEDRO",POLICE OFFICER,POLICE,F,Salary,NaN,87006.0,NaN
14879,"KANE, PAUL A",CAPTAIN,POLICE,F,Salary,NaN,136170.0,NaN
29562,"THURMOND, HUGO E",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN
31067,"WALKER, STACEY D",POLICE OFFICER,POLICE,F,Salary,NaN,96060.0,NaN
7943,"DYSON, DAPHNE M",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
23449,"POLLARD, SHIRLEY",TITLE V PROGRAM TRAINEE I,FAMILY & SUPPORT,P,Hourly,20.0,NaN,8.25
16875,"LINDAHL, CHRISTOPHER A",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [40]:
# Your code here:
na_in_each_column = salaries.isna().sum()
na_in_each_column


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

In [41]:
total_na = salaries.isna().sum().sum()
total_na

58344

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [42]:
hvss_count = salaries.groupby("Salary or Hourly")["Salary or Hourly"].count()
hvss_count


Salary or Hourly
Hourly     8022
Salary    25161
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [43]:
departments_count = salaries.groupby("Department")["Department"].count()
departments_count

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [44]:
import numpy as np
from scipy.stats import ttest_1samp
from statsmodels.stats.weightstats import ttest_ind

In [45]:
#subdataset for hourly employees
hourly_employees = salaries[salaries["Salary or Hourly"] == "Hourly"]
hourly_employees1 = hourly_employees[["Hourly Rate"]]
hourly_employees1

,Hourly Rate
4,19.86
6,46.10
7,35.60
10,2.65
18,17.68
...,...
33164,46.10
33168,17.68
33169,35.60
33174,46.35


In [46]:
hourly_mean = hourly_employees1.mean()

t_statistic, p_value = ttest_1samp(hourly_employees1, hourly_mean)
print(p_value)

if p_value < 0.05:
   print('Reject Null Hypothesis: Hourly workers is 30€/hr')
else:
   print('Accept Null Hypothesis: Hourly workers is different from 30€/hr')

[1.]
Accept Null Hypothesis: Hourly workers is different from 30€/hr


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [47]:
from statsmodels.stats.weightstats import ztest as ztest 

police_salary = salaries[(salaries["Department"] == "POLICE") & (salaries["Salary or Hourly"] == "Salary")]
salary = police_salary["Annual Salary"]
salary

# Conduct the z-test
t_statistic, p_value = ztest(salary, value=86000, alternative="larger")

# Display the results
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

if p_value < 0.05:
   print('Reject Null Hypothesis: This year the salaries are lower or equal to last years salaries')
else:
   print('Accept Null Hypothesis: This year the salaries are higher than last years salaries')

T-statistic: 3.081997005712994
P-value: 0.0010280845287547855
Reject Null Hypothesis: This year the salaries are lower or equal to last years salaries


Using the `crosstab` function, find the department that has the most hourly workers. 

In [48]:
# Cross tabulation of Salary and Hourly
cross_tab = pd.crosstab(salaries["Department"], salaries["Salary or Hourly"])["Hourly"]
print("The department is:",cross_tab.idxmax())
print("This department has:",cross_tab.max(),"hourly workers")

The department is: STREETS & SAN
This department has: 1862 hourly workers


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [49]:
streets_san = salaries[(salaries["Department"] == "STREETS & SAN") & (salaries["Salary or Hourly"] == "Hourly")]
hourly_wage = 35
streets_san
streets_san_hourly = streets_san["Hourly Rate"]
streets_san_hourly

t_statistic, p_value = ttest_1samp(streets_san_hourly, hourly_wage, alternative="less")
print(p_value)

if p_value < 0.05:
    print('Reject Null Hypothesis: Hourly workers is 35€/hr')
else:
    print('Accept Null Hypothesis: Hourly workers is less than 35€/hr')


1.6689265282353859e-21
Reject Null Hypothesis: Hourly workers is 35€/hr


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [51]:
import numpy as np
import pandas as pd
import scipy.stats as st

hourly_employees = salaries[salaries["Salary or Hourly"] == "Hourly"]
hourly_employees1 = hourly_employees[["Hourly Rate"]]
mean_hourly = hourly_employees1.mean()

confidence_level = 0.95
degrees_of_freedom = len(hourly_employees1) - 1

confidence_interval = st.t.interval(alpha=confidence_level, df=degrees_of_freedom, loc=mean_hourly, scale=st.sem(hourly_employees1))
confidence_interval


TypeError: rv_generic.interval() missing 1 required positional argument: 'confidence'

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm

hourly_employees = salaries[salaries["Salary or Hourly"] == "Hourly"]
number_hourly_employees = hourly_employees["Name"].count(); 
total_employees = salaries["Name"].count(); 
null_hypothesis = 0.25

z_stat, p_value = sm.stats.proportions_ztest(number_hourly_employees, total_employees, null_hypothesis)

alpha = 0.05  # 95% confidence level
if p_value < alpha:
    print(f"Reject the null hypothesis. The number of hourly workers is significantly different from 25%.")
else:
    print(f"Fail to reject the null hypothesis. The number of hourly workers is not significantly different from 25%.")

Reject the null hypothesis. The number of hourly workers is significantly different from 25%.
